In [131]:

# for data manipulation
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt
import seaborn as sns
# make a nice white grid when plotting
sns.set_style("whitegrid")


In [99]:
df_train = pd.read_csv('/Users/brandonowens/Downloads/cbg_no2020_gt3crashes_feature_select_and_transform_train.csv')
df_train.head()

,census_block_group,CountHU,Pct_AO0,Pct_AO1,Pct_AO2p,D3A,D3AAO,D3AMM,D3BAO,D3BMM3,D3BMM4,D3BPO3,D3BPO4,D4B025,D4B050,D5AR,D5CRI,NatWalkInd,log_crash_per_density
0,11656,2.713491,-0.941763,0.289673,0.596977,1.325384,-3.000000,0.739317,-2.000000,0.987100,1.287906,63.032625,14.545990,0.0,0.0,283581.0,0.518476,10.333333,0.877227
1,42938,3.091315,-3.000000,0.310526,0.689474,0.460360,-0.771369,-0.019542,-0.348060,0.037986,-1.694607,2.621897,0.295856,0.0,0.0,382.0,0.000000,5.500000,3.442839
2,169884,2.790988,-0.916025,0.334025,0.545643,0.363566,-0.252747,-1.533631,0.119856,-2.000000,-2.000000,1.743760,0.186831,0.0,0.0,9935.0,0.557457,2.500000,2.077353
3,198338,2.859138,-1.000181,0.194444,0.706597,0.344122,-2.813679,-0.354128,-1.186518,0.087026,-0.692936,1.900439,0.165256,0.0,0.0,2232.0,0.042970,3.000000,2.908564
4,6402,2.815578,-0.699033,0.512864,0.288165,1.486949,0.308425,0.307001,1.195819,1.496710,-2.000000,250.993067,0.000000,0.0,0.0,437784.0,0.800408,15.000000,-0.242799


In [100]:
# Get rid of census block column and normalized crashes column.
features = df_train.columns[1:-1]

# single out the target variable
target = df_train.columns[-1]

## Fitting some models

In [101]:
# import our performance metric for the models
from sklearn.metrics import root_mean_squared_error

# create an empty dictionary to store rmses for the different models
model_rmses= {}


In [102]:
# We'll do 5-fold cv to get a good idea of the performance for most of our models 
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=831)



In [103]:
# import our first model
from sklearn.linear_model import LinearRegression
mlr = LinearRegression()

# empty list to hold out rmse at each fold
mlr_rmses = []

for i, (train_index, test_index) in enumerate(kfold.split(df_train)):
    df_tt = df_train.iloc[train_index]
    df_ho = df_train.iloc[test_index]

    # train base model
    mlr.fit(df_tt[features], df_tt[target])

    # predict on holdout set
    mlr_pred = mlr.predict(df_ho[features])

    # get the rmses
    mlr_rmses.append(root_mean_squared_error(df_ho[target], mlr_pred))

print(f"Base MLR Cross-validation RMSE: {np.mean(mlr_rmses)}")





Base MLR Cross-validation RMSE: 0.6313197016071037


In [104]:
# train mlr model on the whole training set and see what the rmse is

mlr.fit(df_train[features], df_train[target])
mlr_rmse = root_mean_squared_error(df_train[target], mlr.predict(df_train[features]))
print(mlr_rmse)

0.6311417674918878


This is basically the same as what we saw with cross validation. We'll take it as our performance score for this model.

In [105]:
model_rmses['mlr'] = round(mlr_rmse,3)


In [106]:
model_rmses

{'mlr': 0.631}

## Let's also look at Ridge and Lasso Regression

In [107]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

alphas = [0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]

# arrays to store rmses for each of the folds at each value of alpha
ridge_rmses = np.zeros((len(alphas), 5))
lasso_rmses = np.zeros((len(alphas), 5))

for i, alph in enumerate(alphas):
    for j, (train_index, test_index) in enumerate(kfold.split(df_train)):

        df_tt = df_train.iloc[train_index]
        df_ho = df_train.iloc[test_index]

        # set up the pipelines
        ridge_pipe = Pipeline([('scale', StandardScaler()),
                       ('ridge', Ridge(alpha=alph))])

        lasso_pipe = Pipeline([('scale', StandardScaler()),
                       ('lasso', Lasso(alpha=alph))])
        
        # fit the models
        ridge_pipe.fit(df_tt[features], df_tt[target])
        lasso_pipe.fit(df_tt[features], df_tt[target])

        # make predictions
        ridge_preds = ridge_pipe.predict(df_ho[features])
        lasso_preds = lasso_pipe.predict(df_ho[features])

        # record the rmses
        ridge_rmses[i,j] = root_mean_squared_error(df_ho[target], ridge_preds)
        lasso_rmses[i,j] = root_mean_squared_error(df_ho[target], lasso_preds)
    


        
print('Best performing ridge regression model used an alpha value of alpha =',alphas[np.argmin(np.mean(ridge_rmses, axis=1))], 'with an average rmse of',np.mean(ridge_rmses, axis=1)[np.argmin(np.mean(ridge_rmses, axis=1))])
print('Best performing lasso regression model used an alpha value of alpha =',alphas[np.argmin(np.mean(lasso_rmses, axis=1))], 'with an average rmse of',np.mean(lasso_rmses, axis=1)[np.argmin(np.mean(lasso_rmses, axis=1))])




Best performing ridge regression model used an alpha value of alpha = 1 with an average rmse of 0.6313196988905494
Best performing lasso regression model used an alpha value of alpha = 1e-05 with an average rmse of 0.6313197158843044


We'll take these as our performance scores for these two models.

In [108]:
# Now we fit the tuned models to the training data and collect the rmses into our performance dictionary
# set up the pipelines
ridge_pipe = Pipeline([('scale', StandardScaler()),
                       ('ridge', Ridge(alpha=1))])

lasso_pipe = Pipeline([('scale', StandardScaler()),
                       ('lasso', Lasso(alpha=0.00001))])
        
# fit the models
ridge_pipe.fit(df_train[features], df_train[target])
lasso_pipe.fit(df_train[features], df_train[target])

# make predictions
ridge_preds = ridge_pipe.predict(df_train[features])
lasso_preds = lasso_pipe.predict(df_train[features])

# add the rmses to the dictionary
model_rmses['ridge'] = round(root_mean_squared_error(df_train[target], ridge_preds),3)
model_rmses['lasso'] = round(root_mean_squared_error(df_train[target], lasso_preds),3)

In [109]:
model_rmses

{'mlr': 0.631, 'ridge': 0.631, 'lasso': 0.631}

RMSEs for both Ridge and Lasso for various values of alpha dont seem to be much better than the base multiple linear regression.

## Let's look a some more complex models. We'll start with Random Forest Regressor.

In [95]:
# Let's fit an out-of-the-box random forest model to our data
from sklearn.ensemble import RandomForestRegressor
# instantiating the model
oob_rfr = RandomForestRegressor(random_state=831)



First we'll get a performance score for out-of-box rrf without cross validation and then we'll use 5-fold cv to get a better sense as to how the out-of-box rrf performs on 'unseen' data.

In [94]:
oob_rfr.fit(df_train[features],df_train[target])
print(round(root_mean_squared_error(df_train[target], oob_rfr.predict(df_train[features])),3))


0.209


That's pretty low. Let's try 5-fold cv and take the mean rmse.

In [55]:
# empty list to hold our rmse at each fold
oob_rfr_rmses = []

for i, (train_index, test_index) in enumerate(kfold.split(df_train)):
    df_tt = df_train.iloc[train_index]
    df_ho = df_train.iloc[test_index]

    # train base model
    oob_rfr.fit(df_tt[features], df_tt[target])

    # predict on holdout set
    oob_rfr_pred = oob_rfr.predict(df_ho[features])

    # get the rmses
    oob_rfr_rmses.append(root_mean_squared_error(df_ho[target], oob_rfr_pred))

print(f"Base RFR mean Cross-validation RMSE: {np.mean(oob_rfr_rmses)}")


Base RFR Cross-validation RMSE: 0.5616972655046627


The difference suggests that our model is overfitting. We need to tune the hyperparameters...

Before that let's just peek at the feature importance scores.

In [42]:
feature_importances = pd.DataFrame({'feature importance score':oob_rfr.feature_importances_}, index = features).sort_values(by='feature importance score', ascending = False)
print(feature_importances)


            feature importance score
D3A                         0.541140
D3BAO                       0.114344
D3BPO3                      0.043505
D5AR                        0.039106
D3AAO                       0.035370
D3BMM3                      0.029134
CountHU                     0.025617
D3BPO4                      0.023588
D3BMM4                      0.023309
D5CRI                       0.023040
D3AMM                       0.022452
NatWalkInd                  0.020036
Pct_AO0                     0.016969
Pct_AO2p                    0.016816
Pct_AO1                     0.016597
D4B050                      0.006589
D4B025                      0.002386


Let's recall: `D3A = Total road network density` and `D3BA0 = Intersection density in terms of auto-oriented intersections
per square mile`. The feature importance scores associated with our out-of-the-box random forest regressor indicate that these are the features in our dataframe with the most impact on the number of crashes (weighted by severity). Is it surprising that D3a is five times as impactful as D3bao? 

Ok, let's now do a grid search to figure out what the optimal max depth and n_estimators could be.

In [117]:

from sklearn.model_selection import GridSearchCV




# The parameter ranges can be changed once we have a better idea of a good range
grid_cv = GridSearchCV(RandomForestRegressor(),
                       param_grid= {'max_depth': range(1,11),
                       'n_estimators': [50,100,150,200]},
                                    scoring = 'neg_mean_squared_error',
                                    cv = 5)

Running the following cell took my computer over 4 hours...

In [63]:
grid_cv.fit(df_train[features], df_train[target])

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': range(1, 11),
                         'n_estimators': [50, 100, 150, 200]},
             scoring='neg_mean_squared_error')

In [64]:
grid_cv.best_params_



{'max_depth': 10, 'n_estimators': 200}

Best hyperparameters for our random forest model are max_depth=10 and n_estimators=200.

In [65]:
pd.DataFrame({'feature importance score':grid_cv.best_estimator_.feature_importances_}, index=features).sort_values(by= 'feature importance score', ascending=False)


,feature importance score
D3A,0.693816
D3BAO,0.135919
D3BPO3,0.035568
D3AAO,0.027882
D5AR,0.024065
D3BMM3,0.017719
D3BMM4,0.011254
D3BPO4,0.009366
CountHU,0.007787
D3AMM,0.007631


Again, it seems like `D3A` is doing most of the heavy lifting. I wonder how correlated that variable is with our target. Let's check.

In [73]:
correlation_matrix = np.corrcoef(df_train['D3A'], df_train['log_crash_per_density'])
print(correlation_matrix)

[[ 1.         -0.73528722]
 [-0.73528722  1.        ]]


hmmm. interesting.

Ok let's see how close the performance score of the tuned model is to the mean cv rmse.

In [70]:
# computing the rmses from the neg mse
np.sqrt(-1*grid_cv.best_score_)

np.float64(0.5759076053115321)

In [130]:
10**0.55

3.548133892335755

If we transform the log-10 normalized target variable back into severity weighted crashes (apply 10^x), then our RMSE comes out to about 3.69 severity weighted crashes. Not too bad.

I'll record the tuned rfr model performance score in our dictionary.


In [110]:

model_rmses['(tuned) rfr'] = 0.58

In [111]:
model_rmses

{'mlr': 0.631, 'ridge': 0.631, 'lasso': 0.631, '(tuned) rfr': 0.58}

## Let's try XGBoost now.

In [112]:
from xgboost import XGBRegressor

# instantiate the models
oob_xgb_reg = XGBRegressor()

We'll do the same thing we did above for the random forest model.

In [113]:
oob_xgb_reg.fit(df_train[features], df_train[target])
print(root_mean_squared_error(df_train[target], oob_xgb_reg.predict(df_train[features])))

0.4778172917137401


Now we do 5-fold cv and compute the mean rmses.

In [115]:
xgb_rmses = []

for i, (train_index, test_index) in enumerate(kfold.split(df_train)):
    df_tt = df_train.iloc[train_index]
    df_ho = df_train.iloc[test_index]

    # train base model
    oob_xgb_reg.fit(df_tt[features], df_tt[target])

    # predict on holdout set
    xgb_pred = oob_xgb_reg.predict(df_ho[features])

    # get the rmses
    xgb_rmses.append(root_mean_squared_error(df_ho[target], xgb_pred))

print(f"Base XGBoost Cross-validation Mean RMSE: {np.mean(xgb_rmses)}")


Base XGBoost Cross-validation Mean RMSE: 0.5577503250283746


So it looks like the out-of-box xgboost model is slightly overfitting the data. Perhaps the model could benefit from some hyperparameter tuning, but it's hard to say that a potential tuned model will do much better than the tuned random forest model from before as far as the performance scores are concerned.

Let's just try a gridsearchCV and see what happens.

In [119]:
xgb_search = GridSearchCV(XGBRegressor(random_state = 831),
                       param_grid= {'learning_rate':[0.01, 0.1, 1],
                       'n_estimators': [5, 10, 15, 20, 50, 100],
                       'max_depth': [1, 3, 5, 7, 10]},
                                    scoring = 'neg_mean_squared_error',
                                    cv = 5)

This cell only took 1 minute to run!! So much better than 4 hours!!

In [120]:
xgb_search.fit(df_train[features], df_train[target])

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=831, ...),
             param_grid={'learning_rate': [0.01, 0.1, 1],
                         'max_depth': [1, 3, 5, 7, 10],
                         'n_estimators': [5, 10, 15, 20, 50, 100]},
             scoring='neg_mean_squared_error')

In [121]:
xgb_search.best_params_

{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}

In [122]:
np.sqrt(-1*xgb_search.best_score_)

np.float64(0.5527012792893988)

So tuning our XGBoost model with `learning_rate = 0.1`, `max_depth = 7`, and `n_estimators = 100` gave us the best performance score of $0.55$. Nice. 

Here's a breakdown of performance scores.

In [123]:
print(pd.DataFrame({'Tuned XGBoost feature importance scores': xgb_search.best_estimator_.feature_importances_}, index = features).sort_values(by='Tuned XGBoost feature importance scores', ascending = False ))

            Tuned XGBoost feature importance scores
D3A                                        0.591192
D3BAO                                      0.148210
D3BPO3                                     0.055457
D3AAO                                      0.034260
D3BMM3                                     0.031657
D4B050                                     0.020009
D5AR                                       0.019845
D3BMM4                                     0.018684
D3BPO4                                     0.018070
D3AMM                                      0.010396
D5CRI                                      0.009499
NatWalkInd                                 0.008735
D4B025                                     0.008429
Pct_AO2p                                   0.007954
CountHU                                    0.006892
Pct_AO0                                    0.006308
Pct_AO1                                    0.004401


So our tuned XGBoost model is performing a tiny bit better than our tuned Random Forest model. Also tuning XBGoost is way faster. My vote is for XGBoost...

I'll record the tuned performance score in our model performance score dictionary.

In [128]:
model_rmses['(tuned) xgb'] = 0.552

In [129]:
model_rmses

{'mlr': 0.631,
 'ridge': 0.631,
 'lasso': 0.631,
 '(tuned) rfr': 0.58,
 '(tuned) xgb': 0.552}

## Now let's try support vector regression

In [79]:
# First let's try support vector regression out of the box
from sklearn.svm import SVR

# Let's also try bagging it for good measure
from sklearn.ensemble import BaggingRegressor

svr1 = SVR() 
bag1 = BaggingRegressor(estimator=svr1,
                       n_estimators=100,
                       bootstrap=True,
                       random_state=831,
                       n_jobs=1)


rmses_base_svr1 = []
rmses_bagged_svr1 = []
maes_base_svr1 = []
maes_bagged_svr1 = []


for i, (train_index, test_index) in enumerate(kfold.split(df_tt)):
    df_ttt = df_tt.iloc[train_index]
    df_ho = df_tt.iloc[test_index]

    #train base model
    svr1.fit(df_ttt[features], df_ttt[target])
    

    #train bagged model
    bag1.fit(df_ttt[features], df_ttt[target])

    # predict base model on holdout set 
    svr1_pred = svr1.predict(df_ho[features])

    # predict bagged model on holdout set
    bag1_pred = bag1.predict(df_ho[features])

    #compute rmses
    rmses_base_svr1.append(root_mean_squared_error(df_ho[target], svr1_pred))
    rmses_bagged_svr1.append(root_mean_squared_error(df_ho[target], bag1_pred))

    #compute maes
    maes_base_svr1.append(mean_absolute_error(df_ho[target], svr1_pred))
    maes_bagged_svr1.append(mean_absolute_error(df_ho[target], bag1_pred))


print(f"Base SVR Cross-validation RMSE: {np.mean(rmses_base_svr1)}")
print(f"Bagged Cross-Validation RMSES: {np.mean(rmses_bagged_svr1)}")
print(f"Base SVR Cross-validation MAES: {np.mean(maes_base_svr1)}")
print(f"Bagged Cross-Validation MAES: {np.mean(maes_bagged_svr1)}")





Base SVR Cross-validation RMSE: 0.8726367220016333
Bagged Cross-Validation RMSES: 0.8726105659153512
Base SVR Cross-validation MAES: 0.7004752410618638
Bagged Cross-Validation MAES: 0.7004718785293711


That took more than 10 hours... Didn't perform too well either...

Ok so it seems like the best performing model is XGBoost with some of the hyperparameters tuned to better fit the patterns in our training data. We will fix these hyperparameters and use this version of xgboost as our final model. Let's now go ahead and evaluate the performance of our final model on the test data.

## Evaluating final model performance on test data


In [135]:
# load the test data
df_test = pd.read_csv('/Users/brandonowens/Downloads/cbg_no2020_gt3crashes_feature_select_and_transform_test.csv')


In [137]:
df_test.head()

,census_block_group,CountHU,Pct_AO0,Pct_AO1,Pct_AO2p,D3A,D3AAO,D3AMM,D3BAO,D3BMM3,D3BMM4,D3BPO3,D3BPO4,D4B025,D4B050,D5AR,D5CRI,NatWalkInd,log_crash_per_density
0,160619,2.725095,-1.874659,0.404321,0.583333,0.451336,-0.119796,-0.233785,0.441106,-0.260991,-0.887330,0.897146,0.179429,0.0,0.0,3126.0,0.201899,3.833333,1.755828
1,109748,3.003891,-1.474263,0.379451,0.587996,1.246941,0.378899,0.021334,0.668149,0.889625,-2.000000,80.556008,13.942386,0.0,0.0,146592.0,0.696141,16.166667,-0.363275
2,44117,2.680336,-0.726274,0.149451,0.663736,1.044391,0.370558,0.403975,0.763424,0.830264,-2.000000,27.019738,5.789944,0.0,0.0,19424.0,0.692848,9.166667,1.374229
3,159053,2.798651,-1.199330,0.271686,0.666121,0.676810,-1.299672,0.113250,-2.000000,0.649013,-2.000000,10.106125,0.808490,0.0,0.0,27042.0,0.334885,11.333333,0.859253
4,72833,3.037028,-1.072766,0.604227,0.312200,1.302243,0.260754,0.567624,0.758277,1.602725,0.633591,104.419514,14.304043,0.0,0.0,21155.0,0.135227,11.333333,0.334751


In [136]:
# predicting on the test data and getting performance score
final_model_rmse = root_mean_squared_error(df_test[target], xgb_search.best_estimator_.predict(df_test[features]))
print(final_model_rmse)

0.5470285685148585


So the final model's performance score on the test data is $0.547$ while its average performance score on the training data was $0.552$. This means that that our final model predicted equally as well on the training and test data, more or less. Hopefully, this suggests that our training data was a very good representation of the sample space, and not that we had some sort of data leakage...